# The two-body problem: a gravitational system and its numerical solution

https://levelup.gitconnected.com/the-two-body-problem-in-python-6bbe4a0b2f88

## The problem

In this notebook we will assess the two-body problem, consisting in two massive objects interacting via gravitational forces.

We want to know their kinematics, i.e. their trajectories and their velocities, as they travel together in space.

### From Newton to Newton

If $A$ and $B$ are our objects (which we can of course think of as planets), we know that the gravitational force they exert on one another is, according to [Newton](https://en.wikipedia.org/wiki/Newton%27s_law_of_universal_gravitation),

$$
F_G = G \frac{m_Am_B}{r^2}
$$

where $r$ is the distance that sets them apart, i.e. $r = |\vec{r}_B - \vec{r}_A|$.

The quantity above is just a number, but like all other forces we should actually express it in **vector form**: we then have two opposite forces acting respectively on $A$ and $B$,

$$
\vec{F}^G_A = G \frac{m_Am_B}{|\vec{r}_B - \vec{r}_A|^2} \vec{u}_{BA}
$$
and
$$
\vec{F}^G_B = G \frac{m_Am_B}{|\vec{r}_B - \vec{r}_A|^2} \vec{u}_{AB},
$$

with $\vec{u}_{BA}$ the unit-length vector directed from $B$ to $A$, which can be written as
$$
\vec{u}_{BA} = \frac{\vec{r}_B - \vec{r}_A}{|\vec{r}_B - \vec{r}_A|}.
$$
In light of the above, $\vec{u}_{AB} = - \vec{u}_{BA}$ so that $\vec{F}^G_B = - \vec{F}^G_A$.


Sir Isaac also tells us, in its [second law of motion](https://en.wikipedia.org/wiki/Newton%27s_laws_of_motion#Second_law), that the vector sum of all the forces acting on a rigid body gives

$$
\vec{F} = m\vec{a};
$$

if we then assume our two bodies to freely travel in space, only being subjected to their reciprocal gravitational pull (which implies $\vec{F} = \vec{F_G}$), we can conclude
$$
G \frac{m_Am_B}{|\vec{r}_B - \vec{r}_A|^3} (\vec{r}_B - \vec{r}_A) = m\vec{a}_A
$$
and 
$$
G \frac{m_Am_B}{|\vec{r}_B - \vec{r}_A|^3} (\vec{r}_A - \vec{r}_B) = m\vec{a}_B.
$$

## Diff eqs 101

The idea is to solve the two equations above to get an expression for $\vec{r}_{A}$ and $\vec{r}_{B}$.

This problem is an example of **differential equation**, where:
- the variables to solve for are functions (the $\vec{r}_{A/B}(t)$ are functions of time, which we ignored so far)
- that same functions occur in the equation as themselves or as one of its derivatives.

We remember from our Physics 1 class that the acceleration is, in fact, a derivative of the position:
$$
\vec{a} = \frac{\partial \vec{v}}{\partial t} = \frac{\partial^2 \vec{r}}{\partial t^2},
$$
and we may want to use the *dot notation* loved by physicists all around, where the time derivative is expressed with a dot over the function: $\frac{\partial \vec{f}}{\partial t} = \dot{f}$.

We can do this without causing any confusion because our functions have only one variable (time), or in other words they are **ordinary differential equations** (ODEs).

To conclude, our final system is
$$
    \begin{cases}
        G \frac{m_Am_B}{|\vec{r}_B - \vec{r}_A|^3} (\vec{r}_B - \vec{r}_A) = m\ddot{\vec{r}}_A \quad & \text{for body $A$}, \\
        \\
        G \frac{m_Am_B}{|\vec{r}_B - \vec{r}_A|^3} (\vec{r}_A - \vec{r}_B) = m\ddot{\vec{r}}_B \quad & \text{for body $B$}.
    \end{cases}   
$$

These are 6 scalar (one-dimensional) equations, since they can be separated in their $x$, $y$ and $z$ components.

A system of ODEs like ours is usually stated as a **Cauchy's problem**: if we have derivatives of our variables up to the nth order, Cauchy requires us to specify initial values (for $t=0$) for all except the last, nth order:
$$
\vec{r}_A(0), \quad \vec{r}_B(0), \quad \dot{\vec{r}}_A, \quad \dot{\vec{r}}_B.
$$

Cauchy didn't require this on a whim: it is actually necessary in order to obtain unique solutions for the system of equations (can you think why?).

### A common trick

We have to take one last step before solving the problem at hand: **lowering the order of the system**, that is the highest order of derivatives inside the equations.

Let's rewrite the system by reintroducing the velocities as derivatives of the position:

$$
    \begin{cases}
        G \frac{m_Am_B}{|\vec{r}_B - \vec{r}_A|^3} (\vec{r}_B - \vec{r}_A) = m\dot{\vec{v}}_A \quad & \text{for body $A$}, \\
        \\
        G \frac{m_Am_B}{|\vec{r}_B - \vec{r}_A|^3} (\vec{r}_A - \vec{r}_B) = m\dot{\vec{v}}_B \quad & \text{for body $B$}.
    \end{cases}   
$$

Surely this expression is equivalent to before, since $v = \dot{r}$; what we need to do, then, is just carry this information with us in the system by appending 
$$
    \begin{cases}
        \vec{v}_A = \dot{\vec{r}}_A \quad & \text{for body $A$}, \\
        \\
        \vec{v}_B = \dot{\vec{r}}_B \quad & \text{for body $B$}.
    \end{cases}   
$$

Nothing really changed in terms of information, since we can always substitute back the last two equations in the first two and get to the original problem, but for computational reasons this trick will reveal itself to be very useful.

In conclusion, our problem ends up becoming a system of ((2 + 2) * 3 = ) **12 first-order ODEs**, and 12 initial conditions to be specified.

### Numerical solving with Euler

While many ODEs can be solved analytically, we are interested in a numerical approximate method that could solve, in principle, any system defined as a Cauchy Problem.

The simple but surprisingly effective [Euler method](https://en.wikipedia.org/wiki/Euler_method#) for first-order differential equations (that's why we performed the trick) works as follows:



## Let's code!

### Imports

At the beginning of each notebook, we place the **imports** of the libraries we intend to use.

In our case, we will use `numpy` for handling numbers and vectors, and both `plotly` and `matplotlib` for plots.

In [ ]:
# Magic function: enables interactive plot
%matplotlib widget
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import plotly.express as px


### Initial conditions

Let's start with setting the kinematic initial conditions of our two bodies $A$ and $B$, namely their positions $r_A(0), r_B(0)$ and their velocities $v_A(0), v_B(0)$.

We use 3d arrays to denote the $x$, $y$ and $z$ dimensions respectively.

In [ ]:
# body mA initial conditions
mA = 1e26  # mass (kg)
rA0 = np.array([1E3, 0, 0])  # initial position (km)
vA0 = np.array([10, -20, 10])  # initial velocity (km/s)

# body mB initial conditions
mB = 1e26  # mass (kg)
rB0 = np.array([-1E3, 0, 0])  # initial position (km)
vB0 = np.array([10, 40, 10])  # initial velocity (km/s)

We want to store this information in a more compact and useful way: we define a *state vector* $y(0)$ containing all initial conditions.

We structure it as a 3d tensor or *array*, in computer science terms, so that each dimension encodes independent information.

To do so we use `numpy.stack`, which stacks arrays on top of one another thus creating new dimensions.

In [ ]:
y0 = np.stack([np.stack([rA0, vA0]), np.stack([rB0, vB0])])

If we inspect the shape of $y(0)$ we can see that it has indeed 3 dimensions:
- the first can have two values, selecting body $A$ or $B$;
- the second can have two values as well and chooses the kinematic variable, either the position $r$ or the velocity $v$;
- the third can have three values, corresponding to the three dimensions $x$, $y$, and $z$.

In [ ]:
y0.shape

We can play with indices and see what comes out of our tensor; a single element is retrieved by passing all three coordinates needed to identify it, while *slices* or projections are obtained by using a `:` in place of one or more of the coordinates.

Here is $v_A^y(0)$:

In [ ]:
# body=A, variable=v, dimension=y
y0[0, 1, 1]

And here is $r_B(0)$:

In [ ]:
# body=B, variable=r, all dimensions
y0[1, 0, :]

Finally, let's set the value of $G$, the gravitational constant:

In [ ]:
G = 6.67259e-20  # Gravitational constant (km**3/kg/s**2)

### The derivatives


In [ ]:
def two_body_eqm_derivatives(_y, t, _G, _mA, _mB):
    """
    derivatives of the equations of motion describing the two-body system
    t is unused, but we keep it for consistency with scipy requirement
    """
    rA = _y[0, 0, :]
    rB = _y[1, 0, :]

    vA = _y[0, 1, :]
    vB = _y[1, 1, :]

    # magnitude of position vector from rA to rB
    distance = np.linalg.norm(rB - rA)

    # accelerations
    aA = _G * _mA * ((rB - rA) / np.power(distance, 3))
    aB = _G * _mB * ((rA - rB) / np.power(distance, 3))

    derivatives = np.stack([np.stack([vA, aA]), np.stack([vB, aB])])

    return derivatives


### Forward time evolution

In [ ]:
dt = 0.001  # time step (s)
tf = 1E2  # end of simulation (s)

In [ ]:
def evolve(y0, tf, dt, method, params):
    history = []
    yn = y0
    t_axis = np.arange(0, tf, dt)
    for tn in t_axis:
        yn = evolve_one_step(yn, tn, dt, method, params)
        history.append(yn.copy())

    history = np.stack(history, axis=-1)
    return history

### Euler's method

In [ ]:
def evolve_one_step(yn, tn, dt, method, params):
    if method == "euler":
        f = two_body_eqm_derivatives(yn, tn, *params)
        yn += f * dt
    elif method == "rk4":
        f1 = two_body_eqm_derivatives(yn, tn, *params)
        f2 = two_body_eqm_derivatives(yn + f1 * dt / 2, tn + dt / 2, *params)
        f3 = two_body_eqm_derivatives(yn + f2 * dt / 2, tn + dt / 2, *params)
        f4 = two_body_eqm_derivatives(yn + f3 * dt, tn + dt, *params)
        yn += (f1 + 2 * f2 + 2 * f3 + f4) * dt / 6

    return yn

### Running the simulation

Finally, let's run the main `evolve` function:

In [ ]:
history = evolve(y0, tf, dt, "euler", params=(G, mA, mB))

In [ ]:
# if we want to compare with odeint from scipy
#from scipy.integrate import odeint
#history = odeint(two_body_eqm_derivatives, y0, np.arange(0, tf, dt), args=(G, mA, mB))

As we can see, the `history` output looks like the initial state vector $y(0)$ but now it has an additional axis storing time information: it is, indeed, the whole sequence of $y(t)$ for $t=0, \dots, t_f$.

In [ ]:
history.shape

### Plotting the outcomes

First let's extract the trajectories of both bodies from `history`, by remembering how indexing is done on multi-dimensional `numpy` arrays:

In [ ]:
# Trajectories
xA = history[0, 0, 0, :]
yA = history[0, 0, 1, :]
zA = history[0, 0, 2, :]

xB = history[1, 0, 0, :]
yB = history[1, 0, 1, :]
zB = history[1, 0, 2, :]

For example, we can plot a single position coordinate against time.

In [ ]:
px.line(xA)

But we can do better: thanks to the `FuncAnimation` object from `matplotlib` we can display an animated 3d plot, where our two bodies can be seen dancing with one another.

In [ ]:
plt.style.use('dark_background')

fig = plt.figure()
ax = plt.axes(projection='3d')


def animate(frame_num):
    ax.clear()
    ax.plot3D(xA[:frame_num], yA[:frame_num], zA[:frame_num], c='blue')
    ax.scatter(xA[frame_num], yA[frame_num], zA[frame_num], c='blue', marker='o')

    ax.plot3D(xB[:frame_num], yB[:frame_num], zB[:frame_num], c='orange')
    ax.scatter(xB[frame_num], yB[frame_num], zB[frame_num], c='orange', marker='o')

    xm = np.min(np.concatenate([xA, xB]))
    xM = np.max(np.concatenate([xA, xB]))
    ym = np.min(np.concatenate([yA, yB]))
    yM = np.max(np.concatenate([yA, yB]))
    zm = np.min(np.concatenate([zA, zB]))
    zM = np.max(np.concatenate([zA, zB]))

    ax.set(xlim3d=(xm, xM), xlabel='X')
    ax.set(ylim3d=(ym, yM), ylabel='Y')
    ax.set(zlim3d=(zm, zM), zlabel='Z')


anim = FuncAnimation(fig, animate, frames=len(xA), interval=10, repeat=False)
plt.show()